In [1]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer

In [2]:
spSession = SparkSession.builder.master("local").appName("Pyspark-ML").getOrCreate()

In [3]:
df = sc.textFile("weatherAUST.csv")

In [4]:
df.cache()

weatherAUST.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
df.count()

65299

In [6]:
df.take(5)

['Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow',
 '2008-12-01,Albury,13.4,22.9,0.6,4.0,0.0,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,8.0,16.9,21.8,No,No',
 '2008-12-02,Albury,7.4,25.1,0.0,4.0,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,8.0,8.0,17.2,24.3,No,No',
 '2008-12-03,Albury,12.9,25.7,0.0,4.0,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,8.0,2.0,21.0,23.2,No,No',
 '2008-12-04,Albury,9.2,28.0,0.0,4.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,8.0,8.0,18.1,26.5,No,No']

In [7]:
#remove o cabeçalho
df2 = df.filter(lambda x: "Location" not in x)
df2.count()

65298

In [8]:
df2.take(5)

['2008-12-01,Albury,13.4,22.9,0.6,4.0,0.0,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,8.0,16.9,21.8,No,No',
 '2008-12-02,Albury,7.4,25.1,0.0,4.0,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,8.0,8.0,17.2,24.3,No,No',
 '2008-12-03,Albury,12.9,25.7,0.0,4.0,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,8.0,2.0,21.0,23.2,No,No',
 '2008-12-04,Albury,9.2,28.0,0.0,4.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,8.0,8.0,18.1,26.5,No,No',
 '2008-12-05,Albury,17.5,32.3,1.0,4.0,0.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No']

In [9]:
def Limpa(strLIST):
    attList = strLIST.split(",")
    
    MinTempNA = attList[2]
    if MinTempNA == "NA":
        MinTempNA = 0
        
    choveuhoje = attList[21]
    if choveuhoje == "No":
        choveuhoje = 0
    else:
        choveuhoje = 1
    
    choveamanha = attList[22]
    if choveamanha == "No":
        choveamanha = 0
    else:
        choveamanha = 1
        
    #pego só algumas colunas
    linha = Row(LOCATION = attList[1], MINTEMP = float(MinTempNA), MAXTEMP = float(attList[3]), RAINFALL = float(attList[4]),
               WINDSPEED3PM = float(attList[12]), PRESSURE9AM = float(attList[15]), HUMIDITY9AM = float(attList[13]),
               CLOUD3PM = float(attList[18]), CLOUD9AM = float(attList[17]), RAINTODAY = float(choveuhoje),
               RAINTOMORROW = float(choveamanha))
    
    return linha

In [10]:
ch = df2.map(Limpa)
ch.cache()
ch.take(5)

[Row(LOCATION='Albury', MINTEMP=13.4, MAXTEMP=22.9, RAINFALL=0.6, WINDSPEED3PM=24.0, PRESSURE9AM=1007.7, HUMIDITY9AM=71.0, CLOUD3PM=8.0, CLOUD9AM=8.0, RAINTODAY=0.0, RAINTOMORROW=0.0),
 Row(LOCATION='Albury', MINTEMP=7.4, MAXTEMP=25.1, RAINFALL=0.0, WINDSPEED3PM=22.0, PRESSURE9AM=1010.6, HUMIDITY9AM=44.0, CLOUD3PM=8.0, CLOUD9AM=8.0, RAINTODAY=0.0, RAINTOMORROW=0.0),
 Row(LOCATION='Albury', MINTEMP=12.9, MAXTEMP=25.7, RAINFALL=0.0, WINDSPEED3PM=26.0, PRESSURE9AM=1007.6, HUMIDITY9AM=38.0, CLOUD3PM=2.0, CLOUD9AM=8.0, RAINTODAY=0.0, RAINTOMORROW=0.0),
 Row(LOCATION='Albury', MINTEMP=9.2, MAXTEMP=28.0, RAINFALL=0.0, WINDSPEED3PM=9.0, PRESSURE9AM=1017.6, HUMIDITY9AM=45.0, CLOUD3PM=8.0, CLOUD9AM=8.0, RAINTODAY=0.0, RAINTOMORROW=0.0),
 Row(LOCATION='Albury', MINTEMP=17.5, MAXTEMP=32.3, RAINFALL=1.0, WINDSPEED3PM=20.0, PRESSURE9AM=1010.8, HUMIDITY9AM=82.0, CLOUD3PM=8.0, CLOUD9AM=7.0, RAINTODAY=0.0, RAINTOMORROW=0.0)]

In [11]:
#volta para dataframe para funcionar o label encoder
chDF = spSession.createDataFrame(ch)
chDF.cache()
chDF.take(5)

[Row(LOCATION='Albury', MINTEMP=13.4, MAXTEMP=22.9, RAINFALL=0.6, WINDSPEED3PM=24.0, PRESSURE9AM=1007.7, HUMIDITY9AM=71.0, CLOUD3PM=8.0, CLOUD9AM=8.0, RAINTODAY=0.0, RAINTOMORROW=0.0),
 Row(LOCATION='Albury', MINTEMP=7.4, MAXTEMP=25.1, RAINFALL=0.0, WINDSPEED3PM=22.0, PRESSURE9AM=1010.6, HUMIDITY9AM=44.0, CLOUD3PM=8.0, CLOUD9AM=8.0, RAINTODAY=0.0, RAINTOMORROW=0.0),
 Row(LOCATION='Albury', MINTEMP=12.9, MAXTEMP=25.7, RAINFALL=0.0, WINDSPEED3PM=26.0, PRESSURE9AM=1007.6, HUMIDITY9AM=38.0, CLOUD3PM=2.0, CLOUD9AM=8.0, RAINTODAY=0.0, RAINTOMORROW=0.0),
 Row(LOCATION='Albury', MINTEMP=9.2, MAXTEMP=28.0, RAINFALL=0.0, WINDSPEED3PM=9.0, PRESSURE9AM=1017.6, HUMIDITY9AM=45.0, CLOUD3PM=8.0, CLOUD9AM=8.0, RAINTODAY=0.0, RAINTOMORROW=0.0),
 Row(LOCATION='Albury', MINTEMP=17.5, MAXTEMP=32.3, RAINFALL=1.0, WINDSPEED3PM=20.0, PRESSURE9AM=1010.8, HUMIDITY9AM=82.0, CLOUD3PM=8.0, CLOUD9AM=7.0, RAINTODAY=0.0, RAINTOMORROW=0.0)]

In [13]:
#possivel fazer a indexação
#criando um label nas colunas
stringIdx = StringIndexer(inputCol="LOCATION", outputCol = "IDX_LOCATION")
si_model = stringIdx.fit(chDF)
chuvaNormDF = si_model.transform(chDF)
chuvaNormDF.show()

+--------+-------+-------+--------+------------+-----------+-----------+--------+--------+---------+------------+------------+
|LOCATION|MINTEMP|MAXTEMP|RAINFALL|WINDSPEED3PM|PRESSURE9AM|HUMIDITY9AM|CLOUD3PM|CLOUD9AM|RAINTODAY|RAINTOMORROW|IDX_LOCATION|
+--------+-------+-------+--------+------------+-----------+-----------+--------+--------+---------+------------+------------+
|  Albury|   13.4|   22.9|     0.6|        24.0|     1007.7|       71.0|     8.0|     8.0|      0.0|         0.0|         2.0|
|  Albury|    7.4|   25.1|     0.0|        22.0|     1010.6|       44.0|     8.0|     8.0|      0.0|         0.0|         2.0|
|  Albury|   12.9|   25.7|     0.0|        26.0|     1007.6|       38.0|     2.0|     8.0|      0.0|         0.0|         2.0|
|  Albury|    9.2|   28.0|     0.0|         9.0|     1017.6|       45.0|     8.0|     8.0|      0.0|         0.0|         2.0|
|  Albury|   17.5|   32.3|     1.0|        20.0|     1010.8|       82.0|     8.0|     7.0|      0.0|         0.

In [16]:
#CORRELAÇÃO ENTRE VARIAVEIS
for i in chuvaNormDF.columns:
    if not(isinstance(chuvaNormDF.select(i).take(1)[0][0], str)):
        print("Correlação da ChoveAmanha com", i, chuvaNormDF.stat.corr('RAINTOMORROW',i))

Correlação da ChoveAmanha com  MINTEMP 0.12183981523672935
Correlação da ChoveAmanha com  MAXTEMP -0.12212077037281745
Correlação da ChoveAmanha com  RAINFALL 0.22716544458948232
Correlação da ChoveAmanha com  WINDSPEED3PM 0.0729937027105102
Correlação da ChoveAmanha com  PRESSURE9AM -0.15798557124903068
Correlação da ChoveAmanha com  HUMIDITY9AM 0.23495782528763703
Correlação da ChoveAmanha com  CLOUD3PM 0.17138668460006976
Correlação da ChoveAmanha com  CLOUD9AM 0.15201004380026178
Correlação da ChoveAmanha com  RAINTODAY 0.27775172858318625
Correlação da ChoveAmanha com  RAINTOMORROW 1.0
Correlação da ChoveAmanha com  IDX_LOCATION 0.004917586825932478


In [33]:
#convertendo para vetor, no caso vou usar o dense
def transformaVetDense(row):
    obj = (row["RAINTOMORROW"], Vectors.dense([row['CLOUD9AM'], row['MINTEMP'], row['CLOUD3PM'], row['PRESSURE9AM']
                                              , row['MAXTEMP'], row['RAINFALL'], row['HUMIDITY9AM'], row['RAINTODAY'],
                                              row['IDX_LOCATION']]))
    return obj

In [34]:
chuvaRDD = chuvaNormDF.rdd.map(transformaVetDense)
chuvaDF1 = spSession.createDataFrame(chuvaRDD, ['label', 'features'])
chuvaDF1.select('label', 'features').show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[8.0,13.4,8.0,100...|
|  0.0|[8.0,7.4,8.0,1010...|
|  0.0|[8.0,12.9,2.0,100...|
|  0.0|[8.0,9.2,8.0,1017...|
|  0.0|[7.0,17.5,8.0,101...|
|  0.0|[8.0,14.6,8.0,100...|
|  0.0|[1.0,14.3,8.0,100...|
|  0.0|[8.0,7.7,8.0,1013...|
|  1.0|[8.0,9.7,8.0,1008...|
|  0.0|[8.0,13.1,8.0,100...|
+-----+--------------------+
only showing top 10 rows



In [35]:
#separando treino e teste
(treino, teste) = chuvaDF1.randomSplit([0.7,0.3], seed = 123)

In [36]:
treino.count()

45724

In [37]:
teste.count()

19574

In [38]:
#treinamento
linearReg = LinearRegression(maxIter = 10)
modelo = linearReg.fit(treino)

In [39]:
#metricas
print("Coeficientes: " + str(modelo.coefficients))
print("Intercepto: " + str(modelo.intercept))

Coeficientes: [-0.0016738843075472514,0.017683586108298184,0.015906418500862197,-0.007530277883521747,-0.015461671974936664,0.0037111024578567156,0.0030403558031590183,0.09956700286601287,7.409987276025646e-05]
Intercepto: 7.690524978020166


In [40]:
#previsões
predictions = modelo.transform(teste)
predictions.select("features", "prediction").show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|[0.0,-2.3,3.0,102...|-0.08453566093334253|
|[0.0,-1.3,0.0,103...| -0.1619838363440671|
|[0.0,-0.8,0.0,101...|-0.04990565469283...|
|[0.0,-0.6,0.0,102...| -0.1757433256568124|
|[0.0,-0.4,0.0,102...|-0.03046721680427...|
|[0.0,-0.2,1.0,102...|-0.12898740486037497|
|[0.0,0.2,0.0,1024...| -0.2309975696524802|
|[0.0,0.5,0.0,1025...|-0.00981366147174...|
|[0.0,0.5,4.0,1021...|  0.0636398202490902|
|[0.0,0.6,2.0,1023...| 0.01248353292866522|
|[0.0,0.6,2.0,1025...| 0.03299668967501468|
|[0.0,0.6,8.0,1028...| 0.09767719368774141|
|[0.0,0.8,0.0,1017...|-0.04314855689859609|
|[0.0,0.8,1.0,1023...|-0.02495017559312...|
|[0.0,0.9,2.0,1031...|-0.11740052719366645|
|[0.0,1.0,0.0,1029...| -0.2672115439641214|
|[0.0,1.1,1.0,1032...|-0.06272157455361604|
|[0.0,1.2,2.0,1026...|-0.05691484329217644|
|[0.0,1.3,1.0,1020...|-0.02898730421814477|
|[0.0,1.4,0.0,1027...|-0.0783527

In [41]:
#coeficiente de determinação R2
avaliador = RegressionEvaluator(predictionCol = "prediction", labelCol="label", metricName="r2")
avaliador.evaluate(predictions)

0.17629110291315797